# Install and call library

In [8]:
# Install gurobipy library
!pip install gurobipy

In [9]:
# Call gurobipy library
import gurobipy as gp

# Call pandas library
import pandas as pd

# Setup sets and parameters

In [10]:
# Sets
I = 80 # Set of workforce (Labor 1, Labor 2, ..., Labor 80) (80 workers)
J = 7 # Set of weekday (Monday, Tuesday, ..., Sunday) (1 week 7 days)
K = 3 # Set of shift (Morning, Afternoon, Evening) (1 day 3 shift)

# Parameters
demand = [
    [18, 15, 10],  # Monday   = 43
    [20, 20, 10],  # Tuesday  = 50
    [24, 20, 16],  # Wednesday= 60
    [16, 14, 10],  # Thursday = 40
    [18, 17, 10],  # Friday   = 45
    [25, 24, 12],  # Saturday = 61
    [22, 25, 13]   # Sunday   = 60
]


ft = [1]*60 + [0]*20  # 80 workers with 60 full-time, 20 part-time)

# Setup model

In [11]:
# Create a gurobipy model
model = gp.Model("production_planning")

# Decision variable
x = model.addVars(I, J, K, vtype=gp.GRB.BINARY, name="Assigning staff i to day j at shift k")

# Objective function
model.setObjective(gp.quicksum(x[i, j, k] for i in range(I) for j in range(J) for k in range(K)), gp.GRB.MINIMIZE)

# Constraints

# Cons 1
model.addConstrs((gp.quicksum(x[i, j, k] for i in range(I)) >= demand[j][k] for j in range(J) for k in range(K)), name="demand fulfill")

# Cons 2
model.addConstrs((gp.quicksum(x[i, j, k] for k in range(K)) <= 1 for i in range(I) for j in range(J)), name="max 1 shift a day")

# Cons 3
model.addConstrs((gp.quicksum(x[i, j, k] for j in range(J) for k in range(K)) <= 5*ft[i] + 3*(1-ft[i]) for i in range(I)), name="5 days for full-time & 3 days for part-time")

# Cons 4
model.addConstrs((x[i, j, 2] + x[i, j+1, 0] <= 1 for i in range(I) for j in range(J-1)), name="if work on shift 3 today then no work on shift 1 tomorrow") #Bc Python starts from 0,1,2,...

# Cons 5
model.addConstrs((gp.quicksum(x[i, j, 2] for j in range(J)) <= 2 for i in range(I)), name="max 2 night shifts")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [12]:
# Optimize model
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1221 rows, 1680 columns and 6560 nonzeros
Model fingerprint: 0x07f2f6da
Variable types: 0 continuous, 1680 integer (1680 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 359.0000000
Presolve time: 0.02s
Presolved: 1221 rows, 1680 columns, 6560 nonzeros
Variable types: 0 continuous, 1680 integer (1680 binary)

Root relaxation: cutoff, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       359.00000  359.0000

In [13]:
if model.status == gp.GRB.OPTIMAL:
    # Prepare data for the table
    table_data = []

    for i in range(I):
        for j in range(J):
            shifts = [k + 1 for k in range(K) if x[i, j, k].x > 1e-6]
            if shifts:
                for k in shifts:
                    table_data.append({
                        'Staff': i + 1,  # Staff numbers starting from 1
                        'Day': j + 1,    # Days starting from 1
                        'Shift': k,      # Shifts starting from 1
                        'Status': 'Assigned'
                    })
            else:
                # No shifts assigned on this day, so staff is on leave
                table_data.append({
                    'Staff': i + 1,
                    'Day': j + 1,
                    'Shift': 'N/A',
                    'Status': 'Leave'
                })

    # Create DataFrame
    df = pd.DataFrame(table_data)

    # Print the DataFrame
    print(df)

    # Optionally, save to a CSV file
    df.to_csv('staff_assignment.csv', index=False)

else:
    print("No optimal solution found.")

     Staff  Day Shift    Status
0        1    1     1  Assigned
1        1    2   N/A     Leave
2        1    3     1  Assigned
3        1    4   N/A     Leave
4        1    5     2  Assigned
..     ...  ...   ...       ...
555     80    3   N/A     Leave
556     80    4     1  Assigned
557     80    5   N/A     Leave
558     80    6   N/A     Leave
559     80    7   N/A     Leave

[560 rows x 4 columns]
